# Indexierung

## Index-Objekte

Die Index-Objekte von pandas sind für die Achsenbeschriftungen und andere Metadaten, wie den Achsennamen, verantwortlich. Jedes Array oder jede andere Sequenz von Beschriftungen, die ihr bei der Konstruktion einer Serie oder eines DataFrame verwendet, wird intern in einen Index umgewandelt:

In [ ]:
import pandas as pd

obj = pd.Series(range(7), index=pd.date_range("2022-02-02", periods=7))

In [ ]:
obj.index

In [ ]:
obj.index[3:]

Indexobjekte sind unveränderlich (*immutable*) und können daher nicht geändert werden:

In [ ]:
obj.index[1] = '2022-02-03'

Die Unveränderlichkeit macht die gemeinsame Nutzung von Indexobjekten in Datenstrukturen sicherer:

In [ ]:
import numpy as np

labels = pd.Index(np.arange(3))

labels

In [ ]:
obj2 = pd.Series(np.random.randn(3),index=labels)

In [ ]:
obj2

In [ ]:
obj2.index is labels

Um einem Array ähnlich zu sein verhält sich ein Index auch wie eine Menge mit fester Größe:

In [ ]:
data1 = {'Code': ['U+0000', 'U+0001', 'U+0002', 'U+0003', 'U+0004', 'U+0005'],
         'Decimal': [0, 1, 2, 3, 4, 5],
         'Octal': ['001', '002', '003', '004', '004', '005']}
df1 = pd.DataFrame(data1)

In [ ]:
df1

In [ ]:
df1.columns

In [ ]:
'Code' in df1.columns

In [ ]:
'Key' in df1.columns

## Achsenindizes mit doppelten Labels

Anders als Python-Sets kann ein Pandas-Index doppelte Label enthalten:

In [ ]:
data2 = {'Code': ['U+0006', 'U+0007'],
        'Decimal': [6, 7],
        'Octal': ['006', '007']}
df2 = pd.DataFrame(data2)
df12 = df1.append(df2)

df12

Bei der [Auswahl](select-filter.ipynb) von doppelten Bezeichnungen werden alle Vorkommen der betreffenden Bezeichnung ausgewählt:

In [ ]:
df12.loc[1]

In [ ]:
df12.loc[2]

Die Datenauswahl ist einer der Hauptpunkte, der sich bei Duplikaten anders verhält. Die Indizierung eines Labels mit mehreren Einträgen ergibt eine Serie, während einzelne Einträge einen Einzelwert ergeben. Dies kann euren Code komplizierter machen, da der Ausgabetyp der Indizierung je nachdem, ob ein Label wiederholt wird oder nicht, variieren kann. Zudem setzen viele Pandas-Funktionen, wie z.B. `reindex`, voraus, dass Label eindeutig sind. Anhand der Eigenschaft `is_unique` des Index könnt ihr feststellen, ob seine Label eindeutig sind oder nicht:

In [ ]:
df12.index.is_unique

Um doppelte Label zu vermeiden, könnt ihr z.B. `ignore_index=True` verwenden:

In [ ]:
df12 = df1.append(df2, ignore_index=True)

df12

## Einige Indexmethoden und -eigenschaften

Jeder Index verfügt über eine Reihe von Methoden und Eigenschaften für die Mengenlogik, die andere allgemeine Fragen zu den darin enthaltenen Daten beantworten. Im folgenden einige nützliche Methoden und Eigenschaften:

Methode | Beschreibung
:------ | :-----------
`append` | verkettet zusätzliche Indexobjekte, wodurch ein neuer Index entsteht
`difference` | berechnet die Differenz zweier Mengen als Index
`intersection` | berechnet die Schnittmenge
`union` | berechnet die Vereinigungsmenge
`isin` | berechnet ein boolesches Array, das angibt, ob jeder Wert in der übergebenen Sammlung enthalten ist
`delete` | berechnet einen neuen Index, wobei das Element in Index `i` gelöscht wird
`drop` | berechnet einen neuen Index durch Löschen der übergebenen Werte
`insert` | berechnet neuen Index durch Einfügen des Elements in den Index `i`
`is_monotonic` | gibt `True` zurück, wenn jedes Element größer oder gleich dem vorherigen Element ist
`is_unique` | gibt `True` zurück, wenn der Index keine doppelten Werte enthält.
`unique` | berechnet das Array der eindeutigen Werte im Index

## Neuindizierung mit `reindex`

Eine wichtige Methode für Pandas-Objekte ist die Neuindizierung, d.h. die Erstellung eines neuen Objekts mit neu angeordneten Werten, die mit dem neuen Index übereinstimmen. Betrachtet z.B.:

In [ ]:
obj = pd.Series(range(7), index=pd.date_range("2022-02-02", periods=7))

In [ ]:
obj

In [ ]:
new_index = pd.date_range("2022-02-03", periods=7)

In [22]:
obj.reindex(new_index)

2022-02-03    1.0
2022-02-04    2.0
2022-02-05    3.0
2022-02-06    4.0
2022-02-07    5.0
2022-02-08    6.0
2022-02-09    NaN
Freq: D, dtype: float64

`reindex` erstellt einen neuen Index und indiziert den DataFrame neu. Standardmäßig werden Werte im neuen Index, für die es keine entsprechenden Datensätze im DataFrame gibt, zu `NaN`.

Bei geordneten Daten wie Zeitreihen kann es wünschenswert sein, bei der Neuindizierung Werte zu interpolieren oder zu füllen. Die Option `method` ermöglicht dies mit einer Methode wie `ffill`, die die Werte vorwärts füllt:

In [ ]:
obj.reindex(new_index, method='ffill')

Bei einem DataFrame kann `reindex` entweder den (Zeilen-)Index, die Spalten oder beides ändern. Wenn nur eine Sequenz übergeben wird, werden die Zeilen im Ergebnis neu indiziert:

In [ ]:
df1.reindex(range(7))

Die Spalten können mit dem Schlüsselwort `columns` neu indiziert werden:

In [ ]:
encoding = ['Octal', 'Code', 'Description']

df1.reindex(columns=encoding)

### Argumente der Funktion `reindex`

Argument | Beschreibung
:------- | :-----------
`labels` | Neue Sequenz, die als Index verwendet werden soll. Kann eine Index-Instanz oder eine andere sequenzähnliche Python-Datenstruktur sein. Ein Index wird genau so verwendet, wie er ist, ohne dass er kopiert wird.
`axis` | Die neu zu indizierende Achse, entweder `index` (Zeilen) oder `columns` (Spalten). Die Vorgabe ist `index`. Ihr könnt alternativ `reindex(index=new_labels)` oder `reindex(columns=new_labels)` verwenden.
`method` | Interpolationsmethode; `ffill` füllt vorwärts, während `bfill` rückwärts füllt.
`fill_value` | Ersatzwert, der zu verwenden ist, wenn fehlende Daten durch Neuindizierung eingefügt werden. Verwendet `fill_value='missing'` (das Standardverhalten), wenn die fehlenden Bezeichnungen im Ergebnis Nullwerte haben sollen.
`limit` |  Beim Vorwärts- oder Rückwärtsfüllen die maximale Anzahl der zu füllenden Elemente.
`tolerance` | Beim Vorwärts- oder Rückwärtsauffüllen die maximale Größe der Lücke, die bei ungenauen Übereinstimmungen gefüllt werden soll.
`level` |  Einfachen Index auf Ebene von `MultiIndex` abgleichen; andernfalls Teilmenge auswählen.
`copy` | Wenn `True`, werden die zugrunde liegenden Daten immer kopiert, auch wenn der neue Index dem alten Index entspricht; wenn `False`, werden die Daten nicht kopiert, wenn die Indizes gleichwertig sind.

## Achsenindizes umbenennen

Die Achsenbeschriftungen können durch eine Funktion oder ein Mapping umgewandelt werden, um neue, anders beschriftete Objekte zu erzeugen. Ihr könnt auch die Achsen an Ort und Stelle ändern, ohne eine neue Datenstruktur zu erstellen. Hier ist ein einfaches Beispiel:

In [ ]:
df3 = pd.DataFrame(np.arange(12).reshape((3, 4)),
                   index=['Deutsch', 'English', 'Français'],
                   columns=[1,2,3,4])

df3

### Achsenindizes umbenennen mit `map`

Wie `Series` haben auch die Achsenindizes eine `map`-Methode:

In [ ]:
transform = lambda x: x[:2].upper()

df3.index.map(transform)

Ihr könnt den Index zuweisen und den DataFrame an Ort und Stelle ändern:

In [ ]:
df3.index = df3.index.map(transform)

df3

### Achsenindizes umbenennen mit `rename`

Wenn ihr eine umgewandelte Version eures Datensatzes erstellen möchtet ohne das Original zu verändern, könnt ihr `rename` verwenden:

In [ ]:
df3.rename(index=str.lower)

Insbesondere kann `rename` in Verbindung mit einem `dict`-ähnlichen Objekt verwendet werden, das neue Werte für eine Teilmenge der Achsenbeschriftungen liefert:

In [ ]:
df3.rename(index={'DE': 'BE', 'EN': 'DE', 'FR': 'EN'},
           columns={1: 0, 2: 1, 3: 2,  4: 3})

`rename` erspart euch das manuelle Kopieren des DataFrame und die Zuweisung seiner Index- und Spaltenattribute. Wenn ihr einen Datensatz an Ort und Stelle ändern möchtet, übergebt zusätzlich noch `inplace=True`:

In [ ]:
df3.rename(index={'DE': 'BE', 'EN': 'DE', 'FR': 'EN'},
           columns={1: 0, 2: 1, 3: 2,  4: 3},
           inplace=True)

df3

## Hierarchische Indizierung

Die hierarchische Indizierung ist eine wichtige Funktion von pandas, die euch ermöglicht, mehrere Indexebenen auf einer Achse zu haben. Dies bietet euch die Möglichkeit, mit höherdimensionalen Daten in einer niedrigdimensionalen Form zu arbeiten.

Beginnen wir mit einem einfachen Beispiel: Erstellen wir eine Reihe Liste von Listen als Index:

In [ ]:
hits = pd.Series([83080,20336,11376,1228,468],
                 index=[['Jupyter Tutorial',
                         'Jupyter Tutorial',
                         'PyViz Tutorial',
                         'Python Basics',
                         'Python Basics'],
                        ['de', 'en', 'de', 'de', 'en']])

hits

Was ihr seht, ist eine graphische Ansicht einer Serie mit einem [pandas.MultiIndex](https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.html). Die „Lücken“ in der Indexanzeige bedeuten, dass die Beschriftung darüber verwendet werden soll.

In [ ]:
hits.index

Bei einem hierarchisch indizierten Objekt ist eine so genannte partielle Indizierung möglich, mit der ihr Teilmengen der Daten gezielt auswählen könnt:

In [ ]:
hits['Jupyter Tutorial']

In [ ]:
hits['Jupyter Tutorial':'Python Basics']

In [ ]:
hits.loc[['Jupyter Tutorial', 'Python Basics']]

Die Auswahl ist sogar von einer „inneren“ Ebene aus möglich. Im folgenden wähle ich alle Werte mit dem Wert `1` aus der zweiten Indexebene aus:

In [ ]:
hits.loc[:, 'de']

Bei einem DataFrame kann jede Achse einen hierarchischen Index haben:

In [ ]:
version_hits = [[19651,0,30134,0,33295,0],
                [4722,1825,3497,2576,4009,3707],
                [2573,0,4873,0,3930,0],
                [525,0,427,0,276,0],
                [157,0,85,0,226,0]]

df = pd.DataFrame(version_hits,
                  index=[['Jupyter Tutorial',
                          'Jupyter Tutorial',
                          'PyViz Tutorial',
                          'Python Basics',
                          'Python Basics'],
                         ['de', 'en', 'de', 'de', 'en']],
                  columns=[['12/2021', '12/2021',
                            '01/2022', '01/2022', 
                            '02/2022', '02/2022'],
                           ['latest', 'stable',
                            'latest', 'stable',
                            'latest', 'stable']])

df

Die Hierarchieebenen können Namen haben (als Zeichenketten oder beliebige Python-Objekte). Wenn dies der Fall ist, werden diese in der Konsolenausgabe angezeigt:

In [ ]:
df.index.names = ['Title', 'Language']
df.columns.names = ['Month', 'Version']

df

<div class="alert alert-block alert-warning">

**Warnung:**

Achtet darauf, dass die Indexnamen `Month` und `Version` nicht Teil der Zeilenbezeichnungen (der `df.index`-Werte) sind.
</div>

Mit [pandas.MultiIndex.from_arrays](https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.from_arrays.html) kann selbst ein `MultiIndex` erstellt und dann wiederverwendet werden; die Spalten im vorangehenden DataFrame mit Ebenennamen könnten so erstellt werden:

In [ ]:
pd.MultiIndex.from_arrays([['Jupyter Tutorial',
                            'Jupyter Tutorial',
                            'PyViz Tutorial',
                            'Python Basics',
                            'Python Basics'],
                           ['de', 'en', 'de', 'de', 'en']],
                          names=['Title', 'Language'])

Mit der Teilspaltenindizierung könnt ihr auf ähnliche Weise Spaltengruppen auswählen:

In [ ]:
df.loc['Jupyter Tutorial']

In [ ]:
df.loc['PyViz Tutorial':'Python Basics']

`loc` mit Liste von Tupeln:

In [ ]:
df.loc[[('Jupyter Tutorial', 'de'), ('PyViz Tutorial', 'de')]]

`loc` mit Liste des zweiten Spaltenindex:

In [ ]:
df.loc[:, ['de'], :]

Verwenden von `slice`:

In [ ]:
df.loc[slice('Jupyter Tutorial'), :]

In [ ]:
df.loc[slice('Jupyter Tutorial', 'PyViz Tutorial'), :]

In [ ]:
df.loc[slice('Jupyter Tutorial', 'PyViz Tutorial'), '12/2021']

Verwenden von `slice`, und Listen:

In [ ]:
df.loc[slice('Jupyter Tutorial', 'PyViz Tutorial'), ['de'], :]

## Ansicht vs. Kopie

In Pandas hängt es von der Struktur und den Datentypen des ursprünglichen DataFrame ab, ob ihr einen View erhaltet oder nicht – und ob Änderungen, die an einem View vorgenommen werden, in den ursprünglichen DataFrame zurück übertragen werden.

<div class=\"alert alert-block alert-info\">
    
**Siehe auch:**

* [Returning a view versus a copy](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy)
* [Views and Copies in pandas](https://www.practicaldatascience.org/html/views_and_copies_in_pandas.html)

</div>

### `stack` und `unstack`

Die hierarchische Indizierung spielt eine wichtige Rolle bei der Umformung von Daten und gruppenbasierten Operationen wie der Bildung einer [Pivot-Tabelle](https://de.wikipedia.org/wiki/Pivot-Tabelle). Zum Beispiel könnt ihr diese Daten mit der [pandas.Series.unstack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unstack.html)-Methode in einen DataFrame umordnen:

In [ ]:
hits.unstack()

Die umgekehrte Operation von unstack ist [stack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html):

In [ ]:
hits.unstack().stack()

### Umordnen und Sortieren von Ebenen

Es kann vorkommen, dass ihr die Reihenfolge der Ebenen auf einer Achse neu anordnen oder die Daten nach den Werten in einer bestimmten Ebene sortieren wollt. Die Funktion [pandas.DataFrame.swaplevel](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.swaplevel.html) nimmt zwei Ebenennummern oder -namen entgegen und gibt ein neues Objekt zurück, in dem die Ebenen vertauscht sind (die Daten bleiben jedoch unverändert):

In [ ]:
df.swaplevel('Language', 'Title')

[pandas.DataFrame.sort_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html) hingegen sortiert die Daten nur nach den Werten in einer einzigen Ebene. Beim Vertauschen von Ebenen ist es nicht unüblich, auch `sort_index` zu verwenden, damit das Ergebnis lexikografisch nach der angegebenen Ebene sortiert wird:

In [ ]:
df.sort_index(level=0)

In [ ]:
df.swaplevel(0, 1).sort_index(level=0)

<div class="alert alert-block alert-info">

**Hinweis:**

Die Leistung der Datenauswahl ist bei hierarchisch indizierten Objekten wesentlich besser, wenn der Index lexikografisch sortiert ist, beginnend mit der äußersten Ebene, d.h. dem Ergebnis des Aufrufs von `sort_index(level=0)` oder `sort_index()`.
</div>

### Zusammenfassende Statistiken nach Ebene

Viele deskriptive und zusammenfassende Statistiken für `DataFrame` und `Series` verfügen über eine Ebenenoption, mit der die Ebene angeben können, nach der ihr auf einer bestimmten Achse aggregieren könnt. Betrachtet den obigen `DataFrame`; wir können entweder die Zeilen oder die Spalten nach der Ebene aggregieren wie folgt:

In [ ]:
df.groupby(level='Language').sum()

In [ ]:
df.groupby(level='Month', axis=1).sum()

Intern wird dazu die [pandas.DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)-Maschinerie von Pandas verwendet, die in [Gruppenoperationen](group-operations.ipynb) näher erläutert wird.

### Indizierung mit den Spalten eines DataFrame

Es ist nicht ungewöhnlich, eine oder mehrere Spalten eines DataFrame als Zeilenindex zu verwenden; alternativ könnt ihr den Zeilenindex auch in die Spalten des DataFrame verschieben. Hier ist ein Beispiel-DataFrame:

In [ ]:
data = [['Jupyter Tutorial', 'de', 19651,0,30134,0,33295,0],
        ['Jupyter Tutorial', 'en', 4722,1825,3497,2576,4009,3707],
        ['PyViz Tutorial', 'de', 2573,0,4873,0,3930,0],
        ['Python Basics', 'de', 525,0,427,0,276,0],
        ['Python Basics', 'en', 157,0,85,0,226,0]]
    
df = pd.DataFrame(data)

df

Die Funktion [pandas.DataFrame.set_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) erstellt einen neuen DataFrame, der eine oder mehrere seiner Spalten als Index verwendet:

In [ ]:
df2 = df.set_index([0,1])

df2

Standardmäßig werden die Spalten aus dem DataFrame entfernt, Ihr könnt sie aber auch drin lassen, indem ihr `drop=False` an `set_index` übergebt:

In [ ]:
df.set_index([0,1], drop=False)

[pandas.DataFrame.reset_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) hingegen bewirkt das Gegenteil von `set_index`; die hierarchischen Indexebenen werden in die Spalten verschoben:

In [ ]:
df2.reset_index()